In [22]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.endpoints.instruments import InstrumentsCandles
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import talib
import pandas_ta as ta
import numpy as np
import pandas as pd
import random


In [23]:
def signal_generator(df):
    vwap = df['VWAP']
    rsi = df['RSI'].dropna()

    close_price = df['Close']
    upper_bb = df['UpperBB']
    lower_bb = df['LowerBB']
    average_rsi = rsi.mean()
    bullish = True
    bearish = True
    sell_count = 0
    buy_count = 0
    #print(rsi.values)
    vwap_threshold = 0.0004
    for i in range(len(df)):
        if max(df['Open'][i], df['Close'][i]) >= (1 + vwap_threshold) * df['VWAP'][i]:
            bearish = False
        if min(df['Open'][i], df['Close'][i]) <= (1 - vwap_threshold) * df['VWAP'][i]:
            bullish = False

    if bullish and bearish:
        vwap_signal =  "Neutral"  # Neither Bullish nor Bearish
    elif bullish:
        vwap_signal =  "Bullish"
        buy_count += 1
    elif bearish:
        vwap_signal =  "Bearish"
        sell_count += 1
    else:
        vwap_signal =  "Neutral"

    if average_rsi > 70:
        rsi_signal = "Overbought"
        sell_count += 1
    elif average_rsi < 30:
        rsi_signal = "Oversold"
        buy_count += 1
    else:
        rsi_signal = "Neutral"
        
    bb_threshold = 0.2
    overbought_signal = (df['Close'] > upper_bb).any()
    oversold_signal = (df['Close'] < lower_bb).any()

    if overbought_signal:
        bb_signal = "Overbought"
        sell_count += 1
    elif oversold_signal:
        bb_signal = "Oversold"
        buy_count += 1
    else:
        bb_signal = "Neutral"

    if buy_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Buy"
    elif sell_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Sell"
    else:
        return "-"
    


### 3 - Connect to the market and execute trades

In [24]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
def get_candles(n, currency):
    client = API(access_token)
    request = InstrumentsCandles(instrument=currency, params={"granularity": Gran.M15, "count": n})
    response = client.request(request)
    candles = response['candles']
    return candles

candles = get_candles(3, Pair.EUR_USD)
candles


[{'complete': True,
  'volume': 221,
  'time': '2023-11-13T22:15:00.000000000Z',
  'mid': {'o': '1.07002', 'h': '1.07006', 'l': '1.06976', 'c': '1.06983'}},
 {'complete': True,
  'volume': 418,
  'time': '2023-11-13T22:30:00.000000000Z',
  'mid': {'o': '1.06984', 'h': '1.07020', 'l': '1.06982', 'c': '1.06990'}},
 {'complete': False,
  'volume': 3,
  'time': '2023-11-13T22:45:00.000000000Z',
  'mid': {'o': '1.06991', 'h': '1.06992', 'l': '1.06990', 'c': '1.06990'}}]

In [25]:
def calculate_vwap(data):
    data['VWAP'] = (((data['High'] + data['Low'] + data['Close']) / 3) * data['Volume']).cumsum() / data['Volume'].cumsum()

def calculate_bollinger_bands(data, std_dev=2):
    data['SMA'] = data['Close'].mean()
    std_dev_values = data['Close'].std()
    data['UpperBB'] = data['SMA'] + (std_dev * std_dev_values)
    data['LowerBB'] = data['SMA'] - (std_dev * std_dev_values)
    
def calculate_rsi(data):
    data['RSI'] = talib.RSI(data['Close'])

In [26]:
def trading_job(currency, string_currency):
    candles = get_candles(15, currency)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low', 'Volume'])
    i=0

    for candle in candles:
        dfstream.loc[i, 'Open'] = float(candle['mid']['o'])
        dfstream.loc[i, 'Close'] = float(candle['mid']['c'])
        dfstream.loc[i, 'High'] = float(candle['mid']['h'])
        dfstream.loc[i, 'Low'] = float(candle['mid']['l'])
        dfstream.loc[i, 'Volume'] = float(candle['volume'])
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)
    dfstream['Volume'] = dfstream['Volume'].astype(float)
    calculate_vwap(dfstream)
    calculate_bollinger_bands(dfstream)
    calculate_rsi(dfstream)

    #print(dfstream)
    #print(dfstream)
    signal = signal_generator(dfstream)#
    # EXECUTING ORDERS
    #print("signal: " + str(signal))
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle['mid']['o']))-previous_candleR
    SLSell = float(str(candle['mid']['o']))+previous_candleR

    TPBuy = float(str(candle['mid']['o']))+previous_candleR*SLTPRatio
    TPSell = float(str(candle['mid']['o']))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    #print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == "Sell":
        random_integer = random.randint(-100, 100)
        mo = MarketOrderRequest(instrument=string_currency, units=-2000 + random_integer, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == "Buy":
        random_integer = random.randint(-100, 100)
        mo = MarketOrderRequest(instrument=string_currency, units=2000 + random_integer, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [27]:

currency_pairs = [Pair.AUD_CAD, Pair.AUD_JPY, Pair.AUD_NZD, Pair.AUD_USD,
    Pair.CAD_CHF, Pair.CAD_JPY,
    Pair.EUR_AUD, Pair.EUR_CAD, Pair.EUR_CHF, Pair.EUR_GBP, Pair.EUR_JPY, Pair.EUR_NZD, Pair.EUR_USD,
    Pair.GBP_AUD, Pair.GBP_CAD, Pair.GBP_CHF, Pair.GBP_JPY, Pair.GBP_NZD, Pair.GBP_USD,
    Pair.NZD_CAD, Pair.NZD_CHF, Pair.NZD_JPY, Pair.NZD_USD,
    Pair.USD_CAD, Pair.USD_CHF, Pair.USD_JPY]

list_of_str_currency = ["AUD_CAD", "AUD_JPY", "AUD_NZD", "AUD_USD",
    "CAD_CHF", "CAD_JPY",
    "EUR_AUD", "EUR_CAD", "EUR_CHF", "EUR_GBP", "EUR_JPY", "EUR_NZD", "EUR_USD",
    "GBP_AUD", "GBP_CAD", "GBP_CHF", "GBP_JPY", "GBP_NZD", "GBP_USD",
    "NZD_CAD", "NZD_CHF", "NZD_JPY", "NZD_USD",
    "USD_CAD", "USD_CHF", "USD_JPY"]


In [29]:
# for i in range (len(list_of_str_currency)):
#     trading_job(currency_pairs[i],list_of_str_currency[i])

scheduler = BlockingScheduler()
for i in range (len(currency_pairs)):
    scheduler.add_job(trading_job, 'cron', args = [currency_pairs[i],list_of_str_currency[i]], day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00')
scheduler.start()


Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:16:00 PST)" was missed by 0:01:58.068076
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:31:00 PST)" was missed by 0:01:58.070160
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:31:00 PST)" was missed by 0:01:58.071679
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:31:00 PST)" was missed by 0:01:58.072682
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:31:00 PST)" was missed by 0:01:58.073624
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 16:31:00 PST)" was misse

{'orderCreateTransaction': {'id': '692', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '692', 'requestID': '25144398896288171', 'time': '2023-11-14T01:16:01.595441424Z', 'type': 'MARKET_ORDER', 'instrument': 'GBP_JPY', 'units': '-2096', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '186.151', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '186.328', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '693', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '692', 'requestID': '25144398896288171', 'time': '2023-11-14T01:16:01.595441424Z', 'type': 'ORDER_FILL', 'orderID': '692', 'instrument': 'GBP_JPY', 'units': '-2096', 'requestedUnits': '-2096', 'price': '186.228', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '99892.2626', 'gainQuoteHomeConversionFactor': '0.006556966553', 'lossQ

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was missed by 0:00:01.008875
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was missed by 0:00:01.042838
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was missed by 0:00:01.022218
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was missed by 0:00:01.028258
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was missed by 0:00:01.029894
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 18:01:00 PST)" was misse

{'orderCreateTransaction': {'id': '696', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '696', 'requestID': '43158827602946748', 'time': '2023-11-14T03:16:00.591868083Z', 'type': 'MARKET_ORDER', 'instrument': 'USD_CAD', 'units': '-1943', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.38000', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.38201', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '697', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '696', 'requestID': '43158827602946748', 'time': '2023-11-14T03:16:00.591868083Z', 'type': 'ORDER_FILL', 'orderID': '696', 'instrument': 'USD_CAD', 'units': '-1943', 'requestedUnits': '-1943', 'price': '1.38128', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '99892.2626', 'gainQuoteHomeConversionFactor': '0.720296809126', 'lossQ

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was missed by 0:00:01.073208
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was missed by 0:00:01.097569
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was missed by 0:00:01.098461
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was missed by 0:00:01.105681
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was missed by 0:00:01.120236
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-11-13 19:46:00 PST)" was misse

{'orderCreateTransaction': {'id': '706', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '706', 'requestID': '43158850249461522', 'time': '2023-11-14T04:46:00.581987888Z', 'type': 'MARKET_ORDER', 'instrument': 'AUD_JPY', 'units': '-2007', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '96.584', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '96.707', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '707', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '706', 'requestID': '43158850249461522', 'time': '2023-11-14T04:46:00.581987888Z', 'type': 'ORDER_FILL', 'orderID': '706', 'instrument': 'AUD_JPY', 'units': '-2007', 'requestedUnits': '-2007', 'price': '96.642', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '99893.3627', 'gainQuoteHomeConversionFactor': '0.006560165643', 'lossQuot

In [10]:
da= pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
da = da[:10000]
signals = []

for i in range(0, len(da)):
    
    block = da.iloc[i:i+15].copy()
    calculate_vwap(block)
    calculate_bollinger_bands(block)
    calculate_rsi(block)
    block = block.reset_index(drop=True)
    if not block.empty:
        signal = signal_generator(block)
        signals.append(signal)
x = pd.Series(signals)
x.value_counts()


-       9209
Buy      397
Sell     394
Name: count, dtype: int64

In [8]:
#consider making 65 / 35 RSI + 1 and make above 70 / 30 as + 2 ssss